In [ ]:
 # Replace with your storage account name and access key
storage_account_name = "tokyoolympicdatasham"
storage_account_access_key = "secret"

container_name = "tokyo-olympic-data-container" 
mount_point = "/mnt/tokyo_olympics_data"


# Mount the storage account to Databricks
dbutils.fs.mount(
  source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
  mount_point = mount_point,
  extra_configs = {f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_access_key}
)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-261678734160318>, line 10
      6 mount_point = "/mnt/tokyo_olympics_data"
      9 # Mount the storage account to Databricks
---> 10 dbutils.fs.mount(
     11  source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
     12  mount_point = mount_point,
     13  extra_configs = {f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_access_key}
     14 )

File /databricks/python_shell/dbruntime/dbutils.py:152, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    150 exc.__context__ = None
    151 exc.__cause__ = None
--> 152 raise exc

ExecutionError: An error occurred while calling o417.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/tokyo_olympics_data; nested

In [ ]:
files = dbutils.fs.ls("/mnt/tokyo_olympics_data")
display(files)


path,name,size,modificationTime
dbfs:/mnt/tokyo_olympics_data/raw-data/,raw-data/,0,0
dbfs:/mnt/tokyo_olympics_data/transformed-data/,transformed-data/,0,0


In [ ]:
files_in_rawdata = dbutils.fs.ls("/mnt/tokyo_olympics_data/raw-data")
display(files_in_rawdata)

path,name,size,modificationTime
dbfs:/mnt/tokyo_olympics_data/raw-data/2425b2ca-d976-400e-89ff-b6a491d1d1f9,2425b2ca-d976-400e-89ff-b6a491d1d1f9,16889,1724092376000
dbfs:/mnt/tokyo_olympics_data/raw-data/2af96f1d-4f04-4b82-84be-459b1c5c9c28,2af96f1d-4f04-4b82-84be-459b1c5c9c28,35270,1724092375000
dbfs:/mnt/tokyo_olympics_data/raw-data/6fa92868-570d-41ba-bc6c-ac14d1dd59ea,6fa92868-570d-41ba-bc6c-ac14d1dd59ea,418492,1724092376000
dbfs:/mnt/tokyo_olympics_data/raw-data/96af8465-23af-4ec7-93e6-b94b09a45d22,96af8465-23af-4ec7-93e6-b94b09a45d22,1123,1724092375000
dbfs:/mnt/tokyo_olympics_data/raw-data/afbd3984-b951-4232-b085-f71a814fcba4,afbd3984-b951-4232-b085-f71a814fcba4,2414,1724092376000


In [ ]:
coaches_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyo_olympics_data/raw-data/2425b2ca-d976-400e-89ff-b6a491d1d1f9")
teams_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyo_olympics_data/raw-data/2af96f1d-4f04-4b82-84be-459b1c5c9c28")
athletes_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyo_olympics_data/raw-data/6fa92868-570d-41ba-bc6c-ac14d1dd59ea")
EntriesGender_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyo_olympics_data/raw-data/96af8465-23af-4ec7-93e6-b94b09a45d22")
Medals_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyo_olympics_data/raw-data/afbd3984-b951-4232-b085-f71a814fcba4")


In [ ]:
print("coaches schema")
(coaches_df.printSchema())
print("teams schema")
(teams_df.printSchema())
print("athletes schema")
(athletes_df.printSchema())
print("EntriesGender schema")
(EntriesGender_df.printSchema())
print("Medals schema")
(Medals_df.printSchema())



coaches schema
root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)

teams schema
root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)

athletes schema
root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)

EntriesGender schema
root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)

Medals schema
root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



JOINING DATA FRAMES  <br />

Join coaches_df with teams_df  <br />
athletes_df with teams_df  <br />
EntriesGender_df with coaches_df  <br />
Medals_df with teams_df 

In [ ]:

joined_df = coaches_df.join(teams_df, ["Discipline", "Event", "Country"], "inner")
athletes_teams_df = athletes_df.join(teams_df, ["Discipline", "Country"], "inner")
coaches_gender_df = coaches_df.join(EntriesGender_df, "Discipline", "left")
medals_teams_df = Medals_df.join(teams_df, Medals_df.TeamCountry == teams_df.Country, "inner")


+-----------------+-----+---------+-------------------+---------+
|       Discipline|Event|  Country|               Name| TeamName|
+-----------------+-----+---------+-------------------+---------+
|Artistic Swimming| Duet|Australia|   MONTICO Loredana|Australia|
|Artistic Swimming| Team|Australia|   MONTICO Loredana|Australia|
|Artistic Swimming| Duet|  Austria|   MLADENOVA Albena|  Austria|
|Artistic Swimming| Duet|  Belarus|SVETLICHNAYA Aliona|  Belarus|
|Artistic Swimming| Duet|   Canada|      SZAUDER Gabor|   Canada|
+-----------------+-----+---------+-------------------+---------+
only showing top 5 rows



AGREGATTING DATA FRAMES  <br />
Total Medals by Country: <br />
Count of Coaches by Discipline:  <br />
Average Gender Distribution in Entries:  <br />





In [ ]:
from pyspark.sql.functions import avg, sum, countDistinct


#total medals by country
total_medals_by_country = Medals_df.groupBy("TeamCountry").agg(
    sum("Gold").alias("TotalGold"),
    sum("Silver").alias("TotalSilver"),
    sum("Bronze").alias("TotalBronze"),
    sum("Total").alias("TotalMedals")
)

#Count of Coaches by Discipline
coaches_count = coaches_df.groupBy("Discipline").count()


# Average Gender Distribution in Entries
avg_gender_distribution = EntriesGender_df.groupBy("Discipline").agg(
    avg("Female").alias("AvgFemale"),
    avg("Male").alias("AvgMale"),
    avg("Total").alias("AvgTotal")
)



FILTERING DATAFRAMES  <br />
Filter Athletes by Discipline:  <br /> 
Select Specific Columns:  <br /> 

Data Enrichment  <br /> 
Add Total Medals Column:  <br /> 
Create a Discipline Summary:  <br /> 

 Data Transformation <br /> 
 Convert Column Types: <br />
 Calculate Rank Difference: <br />
 

In [ ]:
#Filtering Athletes by Discipline
athletes_filtered = athletes_df.filter(athletes_df.Discipline == "Swimming")
# Selectes Specific Columns within a medals dataframe
selected_columns = Medals_df.select("TeamCountry", "Gold", "Silver", "Bronze")


In [ ]:
#Add Total Medals Column
enriched_medals_df = Medals_df.withColumn("TotalMedals", Medals_df["Gold"] + Medals_df["Silver"] + Medals_df["Bronze"])

# Create a Discipline Summary
discipline_summary = teams_df.groupBy("Discipline").agg(
    countDistinct("TeamName").alias("NumTeams"),
    countDistinct("Country").alias("NumCountries")
)


In [ ]:
from pyspark.sql.functions import col


# Converting Column Types
transformed_df = EntriesGender_df.withColumn("Female", col("Female").cast("integer"))


# Calculating Rank Difference
medals_with_diff = Medals_df.withColumn("RankDifference", col("Rank by Total") - col("Rank"))


Writing back to Data lake storage


In [ ]:
base_path = "/mnt/tokyo_olympics_data/transformed-data/"

# Writing each DataFrame to its respective path in Parquet format

total_medals_by_country.write.mode("overwrite").parquet(f"{base_path}/total_medals_by_country")
coaches_count.write.mode("overwrite").parquet(f"{base_path}/coaches_count")
avg_gender_distribution.write.mode("overwrite").parquet(f"{base_path}/avg_gender_distribution")
athletes_filtered.write.mode("overwrite").parquet(f"{base_path}/athletes_filtered")
selected_columns.write.mode("overwrite").parquet(f"{base_path}/selected_columns")
enriched_medals_df.write.mode("overwrite").parquet(f"{base_path}/enriched_medals_df")
discipline_summary.write.mode("overwrite").parquet(f"{base_path}/discipline_summary")
transformed_df.write.mode("overwrite").parquet(f"{base_path}/transformed_df")
medals_with_diff.write.mode("overwrite").parquet(f"{base_path}/medals_with_diff")

